In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import multiprocessing
import numpy as np
import nengo
from nengo.utils.ensemble import sorted_neurons

import sys
sys.path.append('../model')

import benchmark

def 𐌈(**kwargs):
    return kwargs

In [ ]:
# IMPORTANT. Set these parameters.
concurrency = 16 # Number of processor cores
resolution = 21
n_granule = 10000

# Uncomment these for a quick test run
#resolution = 3 
#n_granule = 1000

In [ ]:
def get_benchmark_sweep(input_type="white_noise", **kwargs):
    # Generate a different input sweep depending on the "input_type" parameter
    if input_type == "white_noise":
        input_cback = lambda i, n: \
            benchmark.white_noise_input(benchmark.slin(0.1, 10.0)(i, n))
    elif input_type == "pulse":
        input_cback = lambda i, n: \
            benchmark.pulse_input(1.0 - benchmark.slin(0.1, 0.9)(i, n),
                                        benchmark.slin(0.1, 0.9)(i, n))
    else:
        raise RuntimeError("Invalid input_type")

    # Combine the input sweep with the default parameters
    return benchmark.sweep(
        "input_descr", resolution, input_cback,
        [{**kwargs, **{
            "qp_solver_extra_args": {
                "max_iter": 200,
                "n_threads": 1,
            },
        }}]
    )

def get_benchmark_params_single(**kwargs):
    """
    Produces a single 2D sweep over the delays and the input parameter.
    """
    return {
        "sweep": get_benchmark_sweep(**kwargs),
        "n_repeat": 10,
        "n_delays": resolution,
        "concurrency": concurrency,
        "randomize_all": False,
    }

def get_benchmark_params_multi(name, n, cback, **kwargs):
    """
    Produces multiple 2D sweeps while varying another parameter.
    """
    return {
        "sweep": benchmark.sweep(name, n, cback, get_benchmark_sweep(**kwargs)),
        "n_delays": resolution,
        "concurrency": concurrency,
        "n_repeat": 1, # Only one repetition, but randomize all experiments
        "randomize_all": True,
    }


In [ ]:
def run_benchmark_suite(mode, filename_prefix=None, **kwargs):
    INPUT_TYPE_SHORT = {
        "white_noise": "wn",
        "pulse": "pl"
    }

    for input_type in ["pulse", "white_noise"]:
        prefix = "{}_{}{}".format(
            INPUT_TYPE_SHORT[input_type],
            mode, "" if filename_prefix is None else ("_" + filename_prefix))

        benchmark.run_benchmark(
            filename_prefix=prefix,
            **get_benchmark_params_single(
                input_type=input_type,
                mode=mode,
                **kwargs,
            ))

        if input_type == "pulse":
            continue

#        benchmark.run_benchmark(
#            filename_prefix="sweep_q_" + prefix,
#            **get_benchmark_params_multi(
#                "q", 10, lambda i, n: 1 + i, # Test q=1..10
#                input_type=input_type,
#                mode=mode,
#                **kwargs,
#            ))

        benchmark.run_benchmark(
            filename_prefix="sweep_tau_" + prefix,
            **get_benchmark_params_multi(
                "tau", 10, benchmark.slin(10e-3, 100e-3),
                input_type=input_type,
                mode=mode,
                **kwargs,
            ))


#        bias_modes = [
#            "uniform_pcn_intercepts",
#            "realistic_pcn_intercepts",
#        ]
#        if mode == "two_populations_dales_principle":
#            bias_modes += [
#                "jbias_uniform_pcn_intercepts",
#                "jbias_realistic_pcn_intercepts",
#                "exc_jbias_uniform_pcn_intercepts",
#                "exc_jbias_realistic_pcn_intercepts",
#                "inh_jbias_uniform_pcn_intercepts",
#                "inh_jbias_realistic_pcn_intercepts",
#            ]

#        benchmark.run_benchmark(
#            filename_prefix="bias_mode_" + prefix,
#            **get_benchmark_params_multi(
#                "bias_mode", len(bias_modes), lambda i, n: bias_modes[i],
#                input_type=input_type,
#                mode=mode,
#                **kwargs,
#            ))

#        benchmark.run_benchmark(
#            filename_prefix="sweep_pcn_xi_lower_bound_" + prefix,
#            **get_benchmark_params_multi(
#                "pcn_xi_lower_bound", 10, benchmark.slin(-0.95, -0.05),
#                input_type=input_type,
#                mode=mode,
#                bias_mode=None,
#                **kwargs,
#            ))

#        if mode == "two_populations_dales_principle":
#            # Sweep over the solver regularisation
#            benchmark.run_benchmark(
#                filename_prefix="solver_reg" + prefix,
#                **get_benchmark_params_multi(
#                    "solver_reg", 10, benchmark.slog(1e-4, 1e-1),
#                    input_type=input_type,
#                    mode=mode,
#                    **kwargs,
#                ))
#
#            # Test sub-threshold relaxation
#            benchmark.run_benchmark(
#                filename_prefix="solver_relax" + prefix,
#                **get_benchmark_params_multi(
#                    "solver_relax", 2, lambda i, _: bool(i),
#                    input_type=input_type,
#                    mode=mode,
#                    **kwargs,
#                ))

#            golgi_inh_bias_modes = ["none", "recurrent", "lugaro", "recurrent_and_lugaro"]
#
#            # Test using the golgi recurrence/lugaro cells
#            benchmark.run_benchmark(
#            filename_prefix="sweep_golgi_inh_bias_mode_" + prefix,
#                **get_benchmark_params_multi(
#                    "golgi_inh_bias_mode", len(golgi_inh_bias_modes), lambda i, n: golgi_inh_bias_modes[i],
#                    input_type=input_type,
#                    mode=mode,
#                    **kwargs,
#                ))

#        if mode != "direct" and mode != "single_population":
#            # Fetch the number of granule and golgi cells
#            n_granule = kwargs["n_granule"] if "n_granule" in kwargs else 200
#            n_golgi = kwargs["n_golgi"] if "n_golgi" in kwargs else 20
#
#            # Set the total number of cells and sweep over the ratio
#            kwargs_cpy = {**kwargs}
#            kwargs_cpy["n_go_gr_total"] = n_granule + n_golgi
#            kwargs_cpy["n_granule"] = None
#            kwargs_cpy["n_golgi"] = None
#            benchmark.run_benchmark(
#                filename_prefix="sweep_ratio_" + prefix,
#                **get_benchmark_params_multi(
#                    "go_gr_ratio", 10, benchmark.slog(0.01, 0.5),
#                    input_type=input_type,
#                    mode=mode,
#                    **kwargs_cpy,
#                ))
#
        if filename_prefix == "detailed":
            benchmark.run_benchmark(
                filename_prefix="sweep_n_pcn_granule_convergence_" + prefix,
                **get_benchmark_params_multi(
                    "n_pcn_granule_convergence", 10, lambda i, n: 2 * i + 1,
                    input_type=input_type,
                    mode=mode,
                    record_weights=True,
                    **kwargs,
                ))

            benchmark.run_benchmark(
                filename_prefix="sweep_n_golgi_granule_convergence_" + prefix,
                **get_benchmark_params_multi(
                    "n_golgi_granule_convergence", 10, lambda i, n: 2 * i + 1,
                    input_type=input_type,
                    mode=mode,
                    record_weights=False,
                    **kwargs,
                ))

#            benchmark.run_benchmark(
#                filename_prefix="sweep_spatial_sigma_" + prefix,
#                **get_benchmark_params_multi(
#                    "spatial_sigma", 10, benchmark.slog(0.25, 2.0),
#                    input_type=input_type,
#                    mode=mode,
#                    **kwargs,
#                ))

In [ ]:
default_kwargs = 𐌈(
        bias_mode="jbias_uniform_pcn_intercepts",
)

detailed_kwargs = 𐌈(
        bias_mode="jbias_very_realistic_pcn_intercepts",
        use_spatial_constraints=True,
        n_pcn_golgi_convergence=100,
        n_pcn_granule_convergence=5,
        n_granule_golgi_convergence=100,
        n_golgi_granule_convergence=5,
        n_golgi_golgi_convergence=100,
        n_granule=n_granule,
        n_golgi=100,
        pcn_max_rates=(50, 120),
)

In [ ]:
run_benchmark_suite(
        mode="direct",
        **default_kwargs,
)

In [ ]:
run_benchmark_suite(
        mode="single_population",
        **default_kwargs,
)

In [ ]:
run_benchmark_suite(
        mode="two_populations",
        **default_kwargs,
)

In [ ]:
run_benchmark_suite(
        mode="two_populations_dales_principle",
        **default_kwargs,
)

In [ ]:
run_benchmark_suite(
        mode="two_populations_dales_principle",
        filename_prefix="detailed",
        **detailed_kwargs
)